# Validación del modelo mediante consultas en MongoDB

En este notebook se desarrollan una serie de consultas sobre la colección `locales` con el objetivo de validar el modelo documental implementado.

Las consultas permiten:

- Comprobar la coherencia estructural del modelo embebido.
- Analizar el comportamiento de arrays como `licencias`, `terrazas` y `actividad_economica`.
- Verificar la consistencia de los datos tras el proceso de limpieza y carga.
- Evaluar la capacidad del modelo para responder a consultas analíticas reales.

Se utilizarán principalmente pipelines de agregación junto con operadores como `$unwind`, `$group`, `$match` y `$project`, demostrando la flexibilidad del modelo desnormalizado adoptado.


In [ ]:
from pymongo import MongoClient
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)

client = MongoClient("mongodb://localhost:27017")
db = client["actividad_comercial_madrid"]
col = db["locales"]

print("Documentos en colección:", col.count_documents({}))

In [ ]:
def pretty(df):
    """Convierte _id de Mongo en columnas normales sin romper si es string"""
    if "_id" in df.columns:
        # caso: _id es objeto {campo:valor}
        if isinstance(df["_id"].iloc[0], dict):
            id_df = pd.json_normalize(df["_id"])
            df = pd.concat([id_df, df.drop(columns=["_id"])], axis=1)
        else:
            # caso: _id es valor simple
            df = df.rename(columns={"_id": "valor"})
    return df

## Uso del operador $unwind

El modelo de datos implementado es desnormalizado y embebido, por lo que
determinados atributos (licencias, terrazas y actividad económica) se almacenan
como arrays de subdocumentos dentro de cada local.

Sin embargo, algunas consultas requieren analizar cada elemento del array de
forma individual, por ejemplo:

- Contar licencias por tipo
- Filtrar por estado de licencia
- Calcular la actividad económica predominante

Para ello se utiliza el operador `$unwind`, que transforma cada elemento del
array en un documento independiente dentro del pipeline de agregación.

Ejemplo conceptual:

Documento original:
```
Local
└── licencias: [L1, L2, L3]
```


Tras `$unwind`:
```
Local + L1
Local + L2
Local + L3
```

Esto permite aplicar `$group`, `$match` o `$sort` sobre cada elemento como si
fuese una fila independiente, equivalente al comportamiento de una tabla hija
en un modelo relacional.

Por tanto, `$unwind` no altera los datos almacenados, sino que adapta
temporalmente la estructura durante la consulta para poder realizar
operaciones analíticas sobre arrays embebidos.


### Consulta A

El Total de locales y terrazas por distrito y barrio: construye una consulta que 
permita obtener el total de locales y terrazas agrupados por cada distrito y 
barrio.

In [ ]:
pipeline = [    
    {
        "$group": {
            "_id": {
                "Distrito": "$desc_distrito_local",
                "Barrio": "$desc_barrio_local"
            },
            "Total_locales": {"$sum": 1},
            "Total_terrazas": {"$sum": {"$size": "$terrazas"}}
        }
    },
    {"$sort": {"_id.distrito": 1, "_id.barrio": 1}}
]

result = list(col.aggregate(pipeline))
display(pretty(pd.DataFrame(result)).head(5).style.hide(axis='index'))


### Consulta B

Tipos de licencias y cantidad de licencias por cada tipo: crea una consulta 
para contar cuántas licencias hay de cada tipo en los datos.

In [ ]:
pipeline = [
    {"$unwind": "$licencias"},
    {
        "$group": {
            "_id": {"$ifNull": ["$licencias.desc_tipo_licencia", "Desconocido"]},
            "Total": {"$sum": 1}
        }
    },
    {"$sort": {"Total": -1}}
]

result = list(col.aggregate(pipeline))
display(pretty(pd.DataFrame(result)).style.hide(axis='index'))


### Consulta C

Listado de locales y terrazas con licencias “En tramitación”: diseña una consulta 
que filtre y devuelva un listado detallado de locales y terrazas cuyo estado de 
licencia sea “En trámite”.

In [ ]:
pipeline = [
    {"$unwind": "$licencias"},
    {
        "$match": {
            "$expr": {
                "$regexMatch": {
                    "input": {"$toLower": "$licencias.desc_tipo_situacion_licencia"},
                    "regex": "tramitación"
                }
            }
        }
    },
    {
        "$project": {
            "_id": 0,
            "id_local": 1,
            "distrito": "$desc_distrito_local",
            "barrio": "$desc_barrio_local",
            "licencia": "$licencias.desc_tipo_licencia",
            "estado": "$licencias.desc_tipo_situacion_licencia",
            "num_terrazas": {"$size": "$terrazas"}
        }
    },

    {"$sort": {"num_terrazas": -1}}
]

result = list(col.aggregate(pipeline))
display(pretty(pd.DataFrame(result)).head(10).style.hide(axis='index'))


### Consulta D

Consulta por sección, división y epígrafe de la actividad comercial: crea una 
consulta para clasificar locales y terrazas según los campos sección, división y 
epígrafe.

In [ ]:
pipeline = [
    {"$unwind": "$actividad_economica"},
    {
        "$match": {
            "actividad_economica.desc_seccion": {"$ne": None},
            "actividad_economica.desc_division": {"$ne": None},
            "actividad_economica.desc_epigrafe": {"$ne": None}
        }
    },
    {
        "$project": {
            "_id": 0,
            "id_local": 1,
            "seccion": "$actividad_economica.desc_seccion",
            "division": "$actividad_economica.desc_division",
            "epigrafe": "$actividad_economica.desc_epigrafe"
        }
    }
]

result = list(col.aggregate(pipeline))
df = pd.DataFrame(result)
display(df.sort_values(["seccion","division","epigrafe"]).head(10).style.hide(axis='index'))



### Consulta E

Actividad económica más frecuente por barrio y distrito: diseña una consulta 
que identifique cuál es la actividad económica predominante en cada barrio y 
distrito. 

In [ ]:
pipeline = [
    {"$unwind": "$actividad_economica"},
    {
        "$match": {
            "actividad_economica.desc_epigrafe": {
                "$nin": [None, "", "VALOR NULO EN ORIGEN"]
            }
        }
    },
    {
        "$group": {
            "_id": {
                "distrito": "$desc_distrito_local",
                "barrio": "$desc_barrio_local",
                "actividad": "$actividad_economica.desc_epigrafe"
            },
            "conteo": {"$sum": 1}
        }
    },
    {"$sort": {"conteo": -1}},
    {
        "$group": {
            "_id": {
                "barrio": "$_id.barrio",
                "distrito": "$_id.distrito"
                
            },
            "actividad_predominante": {"$first": "$_id.actividad"},
            "frecuencia": {"$first": "$conteo"}
        }
    },
    {"$sort": {"frecuencia": -1}}
]

result = list(col.aggregate(pipeline))
display(pretty(pd.DataFrame(result)).head(15).style.hide(axis='index'))


### Tratamiento de valores nulos

Se ha añadido la siguiente etapa en el pipeline:

```python
{
    "$match": {
        "actividad_economica.desc_epigrafe": {
            "$nin": [None, "", "VALOR NULO EN ORIGEN"]
        }
    }
}
```

El objetivo es evitar que valores como `"VALOR NULO EN ORIGEN"` afecten al cálculo de la actividad económica predominante.

> Es importante destacar que **no se han eliminado estos registros durante la fase de carga**.  
> Esto se debe a que, aunque no se disponga de información específica sobre el epígrafe de la actividad económica, estos registros siguen siendo útiles para el **conteo total de locales que ejercen alguna actividad económica**, aunque se desconozca cuál es exactamente.

De este modo, se preserva la integridad del dataset original, aplicando el filtrado únicamente en el contexto de la consulta.

### Consulta F

Actualización de horarios de apertura y cierre de ciertos locales: modifica los 
horarios de apertura y cierre de un conjunto seleccionado de locales según un 
criterio que tú determines.

In [ ]:
result = col.update_many(
    {
        "$expr": {
            "$gt": [
                {
                    "$sum": {
                        "$map": {
                            "input": "$terrazas",
                            "as": "t",
                            "in": {"$ifNull": ["$$t.mesas_es", 0]}
                        }
                    }
                },
                5
            ]
        }
    },
    {
        "$set": {
            "hora_apertura1": "08:33",
            "hora_cierre1": "02:33"
        }
    }
)

print("Locales modificados:", result.modified_count)

### Verificación

In [ ]:
display(pd.DataFrame(list(col.find(
    {"hora_cierre1": "02:33"},
    {"_id":0,"id_local":1,"hora_apertura1":1,"hora_cierre1":1}
).limit(5))).style.hide(axis='index'))

